# Integer Linear Programming (ILP) Model for the Package-to-Nodes and Service Center Assignment Problem

## Sets
- $I=\{0,1,\dots,N-1\}$: set of delivery nodes.  
- $J=\{0,1,\dots,M-1\}$: set of packages.

## Parameters
- $K_i\in\mathbb{Z}_{+}$: maximum capacity (number of packages) of node $i\in I$.  
- $c_{i}\ge 0$: fixed cost for delivering a package through node $i$.  
- $S\ge 0$: cost of sending a package directly from the Service Center (SC).  
- $a_{ij}\in\{0,1\}$: coverage parameter; $a_{ij}=1$ if node $i$ can cover package $j$, and $0$ otherwise.

## Decision Variables
- $x_{ij}\in\{0,1\}$:

$$
x_{ij}=
\begin{cases}
1 & \text{if package } j \text{ is assigned to node } i,\\
0 & \text{otherwise.}
\end{cases}
$$

## Objective Function
Minimize the total delivery cost:

$$
\min\ \sum_{i\in I}\sum_{j\in J}\,c_{ij}\,x_{ij}\;+\;\sum_{j\in J} S(1- x_{ij})
$$

For convenience, we rewrite it as:
$$
\min\ \sum_{i\in I}\sum_{j\in J}\,(c_{ij}-S)\,x_{ij}\;+\;\sum_{j\in J} S
$$

**Intuition.** $\sum_{j}S$ is the cost of sending *all* packages through the SC; every time we assign a package $j$ to a node $i$, we adjust the cost by adding $(c_{ij}-S)$.

## Constraints

1. **At most one node handles each package:**

$$
\sum_{i\in I} x_{ij}\ \le\ 1 \qquad \forall j\in J
$$

2. **Node capacities must be respected:**

$$
\sum_{j\in J} x_{ij}\ \le\ K_i \qquad \forall i\in I
$$

3. **Assignments must respect coverage feasibility:**

$$
x_{ij}\ \le\ a_{ij} \qquad \forall i\in I,\ \forall j\in J
$$

4. **Assignment variables are binary:**

$$
x_{ij}\in\{0,1\} \qquad \forall i\in I,\ \forall j\in J
$$


## Definition of helper functions to solve the problem:

* **read_instances**: reads the input file and prepares the variables needed to solve the problem.  
* **solve_problem**: defines the objective function and the constraints as specified above, and minimizes the total cost using `pulp.LpProblem`.


In [ ]:
import pulp as pl

def read_instances(input_path):
    """
    Reads a text file with the format specified in the statement and returns a list 
    of dictionaries, each containing one problem instance and its data. 
    """
    instances = []
    with open(input_path, 'r', encoding='utf-8') as f:
        it = iter(line.strip() for line in f if line.strip() != '')
        while True:
            line = next(it, None)
            if line is None:
                break
            N, M = map(int, line.split())   
            if N == 0 and M == 0:
                break

            S = float(next(it))
            capacities, costs = [], []
            for _ in range(N):
                cap_i, cost_i = next(it).split()
                capacities.append(int(cap_i))
                costs.append(float(cost_i))

            coverage = []
            for _ in range(N):
                parts = next(it).split()
                k = int(parts[0])
                packages = list(map(int, parts[1:]))[:k]
                coverage.append(packages)

            instances.append({
                "N": N, "M": M, "S": S,
                "capacities": capacities, "costs": costs,
                "coverage": coverage
            })
    return instances


def solve_problem(inst):
    """
    Takes a dictionary with the relevant variables and optimizes the problem,
    i.e., finds the minimum cost and the corresponding assignments.
    """
    N, M = inst["N"], inst["M"]
    S = inst["S"]
    capacities = inst["capacities"]
    costs = inst["costs"]
    coverage = inst["coverage"]

    I = list(range(N))
    J = list(range(M))

    K = {i: capacities[i] for i in I}
    c = {(i, j): costs[i] for i in I for j in J}
    s = {j: S for j in J}
    a = {(i, j): 1 if j in set(coverage[i]) else 0 for i in I for j in J}

    #create the optimization problem: 
    model = pl.LpProblem("package_assignment", pl.LpMinimize)

    #decision variables x_ij are binary: 
    x = pl.LpVariable.dicts("x", (I, J), lowBound=0, upBound=1, cat="Binary")

    cost_sc = pl.lpSum(s[j] for j in J)
    cost_nodes = pl.lpSum((c[i, j] - s[j]) * x[i][j] for i in I for j in J)
    model += cost_nodes + cost_sc

    #at most one node per package
    for j in J:
        model += pl.lpSum(x[i][j] for i in I) <= 1
    #node capacity constraints
    for i in I:
        model += pl.lpSum(x[i][j] for j in J) <= K[i]
    #coverage feasibility constraints
    for i in I:
        for j in J:
            if a[i, j] == 0:
                model += x[i][j] <= 0

    solver = pl.PULP_CBC_CMD(msg=False)
    model.solve(solver)

    status = pl.LpStatus[model.status]
    if status != "Optimal":
        return {"status": status, "msg": "not optimal"}, None

    cost = pl.value(model.objective)
    assignments = []
    for j in J:
        i_star = next((i for i in I if (pl.value(x[i][j]) or 0) > 0.5), None)
        assignments.append((j, -1 if i_star is None else i_star))
    return cost, assignments


Solving the problem and writing the results into a .txt file containing the total cost and the assignments:


In [ ]:
input_path = "input.txt"
output_path = "problem.out"

instances = read_instances(input_path)
with open(output_path, "w", encoding="utf-8") as out:
    for k, inst in enumerate(instances, 1):
        cost, assignments = solve_problem(inst)  
        print(f"Case {k}", file=out)
        if assignments is None:
            print(f"# Status: {cost['status']}", file=out)
        else:
            print(f"{cost:.2f}", file=out)
            for j, loc in assignments:
                print(f"{j} {loc}", file=out)
        if k < len(instances):
            print("", file=out)
